In [2]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

def calcular_valor_final_y_tabla(deposito_inicial, aporte_periodico, tasa_interes_anual, meses, frecuencia):
    frecuencias = {
        'semanal': 52,
        'mensual': 12,
        'bimestral': 6,
        'trimestral': 4
    }

    if frecuencia not in frecuencias:
        raise ValueError("Frecuencia no válida. Debe ser 'semanal', 'mensual', 'bimestral' o 'trimestral'.")

    periodos_por_ano = frecuencias[frecuencia]
    tasa_interes_periodica = tasa_interes_anual / periodos_por_ano
    total_periodos = int(meses * (periodos_por_ano / 12))

    valor_final = deposito_inicial
    tabla_resultados = []

    for periodo in range(1, total_periodos + 1):
        if periodo > 1:
            valor_final += aporte_periodico
        ganancia = valor_final * (tasa_interes_periodica / 100)
        valor_final += ganancia

        tabla_resultados.append({
            'Periodo': periodo,
            'Aporte ($)': aporte_periodico if periodo > 1 else 0,
            'Capital ($)': valor_final - ganancia,
            'Ganancia ($)': ganancia,
            'Total ($)': valor_final
        })

    return valor_final, tabla_resultados

def solicitar_valores():
    deposito_inicial = float(deposito_inicial_widget.value)
    if deposito_inicial < 50:
        with output_widget:
            clear_output()
            print("El depósito inicial debe ser de al menos $50.")
        return None

    aporte_periodico = float(aporte_periodico_widget.value)
    if aporte_periodico < 5:
        with output_widget:
            clear_output()
            print("El aporte periódico debe ser de al menos $5.")
        return None

    frecuencia = frecuencia_widget.value
    if frecuencia not in ['semanal', 'mensual', 'bimestral', 'trimestral']:
        with output_widget:
            clear_output()
            print("Frecuencia no válida. Debe ser 'semanal', 'mensual', 'bimestral' o 'trimestral'.")
        return None

    meses = int(meses_widget.value)
    if meses <= 0:
        with output_widget:
            clear_output()
            print("El número de meses debe ser mayor a 0.")
        return None

    tasa_interes_anual = float(tasa_interes_anual_widget.value) if tasa_interes_anual_widget.value else None
    if tasa_interes_anual is not None and tasa_interes_anual < 0:
        with output_widget:
            clear_output()
            print("La tasa de interés anual debe ser positiva.")
        return None

    objetivo_valor_final = float(objetivo_valor_final_widget.value) if objetivo_valor_final_widget.value else None

    return deposito_inicial, aporte_periodico, frecuencia, meses, tasa_interes_anual, objetivo_valor_final

def actualizar_grafica(tabla_resultados):
    periodos = [fila['Periodo'] for fila in tabla_resultados]
    totales = [fila['Total ($)'] for fila in tabla_resultados]

    plt.figure(figsize=(10, 6))
    plt.plot(periodos, totales, marker='o')
    plt.title('Evolución del Valor Total en la Cuenta de Ahorros')
    plt.xlabel('Periodo')
    plt.ylabel('Total ($)')
    plt.grid(True)
    plt.show()

def calcular_interes_anual_biseccion(deposito_inicial, aporte_periodico, meses, frecuencia, objetivo_valor_final, tol=1e-6, max_iter=1000):
    frecuencias = {
        'semanal': 52,
        'mensual': 12,
        'bimestral': 6,
        'trimestral': 4
    }
    periodos_por_ano = frecuencias[frecuencia]
    total_periodos = int(meses * (periodos_por_ano / 12))

    def valor_final_con_tasa(tasa_interes_anual):
        tasa_interes_periodica = tasa_interes_anual / periodos_por_ano
        valor_final = deposito_inicial

        for periodo in range(1, total_periodos + 1):
            if periodo > 1:
                valor_final += aporte_periodico
            ganancia = valor_final * (tasa_interes_periodica / 100)
            valor_final += ganancia

        return valor_final

    a, b = 0, 100  # Supongamos que la tasa de interés anual estará entre 0% y 100%
    fa, fb = valor_final_con_tasa(a) - objetivo_valor_final, valor_final_con_tasa(b) - objetivo_valor_final

    if fa * fb > 0:
        raise ValueError("No se puede aplicar el método de bisección: la función no cambia de signo en el intervalo dado.")

    for _ in range(max_iter):
        c = (a + b) / 2
        fc = valor_final_con_tasa(c) - objetivo_valor_final

        if abs(fc) < tol:
            return c

        if fa * fc < 0:
            b, fb = c, fc
        else:
            a, fa = c, fc

    raise ValueError("El método de bisección no convergió en el número máximo de iteraciones.")

def main(b):
    valores = solicitar_valores()
    if valores is None:
        return

    deposito_inicial, aporte_periodico, frecuencia, meses, tasa_interes_anual, objetivo_valor_final = valores

    if tasa_interes_anual is None and objetivo_valor_final is not None:
        tasa_interes_anual = calcular_interes_anual_biseccion(deposito_inicial, aporte_periodico, meses, frecuencia, objetivo_valor_final)
        with output_widget:
            clear_output()
            print(f"La tasa de interés anual calculada para alcanzar un valor final de ${objetivo_valor_final:.2f} es: {tasa_interes_anual:.6f}%")

    if tasa_interes_anual is None:
        with output_widget:
            clear_output()
            print("Debe proporcionar una tasa de interés anual o un objetivo de valor final.")
        return

    valor_final, tabla_resultados = calcular_valor_final_y_tabla(deposito_inicial, aporte_periodico, tasa_interes_anual, meses, frecuencia)

    with output_widget:
        clear_output()
        if imprimir_tabla_widget.value:
            df = pd.DataFrame(tabla_resultados)
            display(df)
        print(f"\nEl valor final en la cuenta de ahorros es: ${valor_final:.2f}")
        print(f"Tasa de interés anual: {tasa_interes_anual:.6f}%")
        actualizar_grafica(tabla_resultados)

# Widgets para el cálculo principal
deposito_inicial_widget = widgets.FloatText(description="Depósito Inicial ($):", value=50)
aporte_periodico_widget = widgets.FloatText(description="Aporte Periódico ($):", value=5)
tasa_interes_anual_widget = widgets.FloatText(description="Tasa de Interés Anual (%):", value=None, placeholder="Dejar en blanco si se proporciona objetivo")
frecuencia_widget = widgets.Dropdown(
    options=['semanal', 'mensual', 'bimestral', 'trimestral'],
    description="Frecuencia:"
)
meses_widget = widgets.IntText(description="Meses:", value=12)
objetivo_valor_final_widget = widgets.FloatText(description="Objetivo Valor Final ($):", value=None, placeholder="Dejar en blanco si se proporciona tasa de interés")
imprimir_tabla_widget = widgets.Checkbox(description="Imprimir Tabla de Resultados", value=True)
calcular_button = widgets.Button(description="Calcular")

output_widget = widgets.Output()

calcular_button.on_click(main)

display(deposito_inicial_widget, aporte_periodico_widget, tasa_interes_anual_widget, frecuencia_widget, meses_widget, objetivo_valor_final_widget, imprimir_tabla_widget, calcular_button, output_widget)

FloatText(value=50.0, description='Depósito Inicial ($):')

FloatText(value=5.0, description='Aporte Periódico ($):')

FloatText(value=0.0, description='Tasa de Interés Anual (%):')

Dropdown(description='Frecuencia:', options=('semanal', 'mensual', 'bimestral', 'trimestral'), value='semanal'…

IntText(value=12, description='Meses:')

FloatText(value=0.0, description='Objetivo Valor Final ($):')

Checkbox(value=True, description='Imprimir Tabla de Resultados')

Button(description='Calcular', style=ButtonStyle())

Output()